# Weaviate

이 노트북에서는 Weaviate 벡터스토어를 시작하는 방법을 다룹니다.

Weaviate는 오픈소스 벡터 데이터베이스로, 대규모 벡터 검색과 복잡한 쿼리를 지원합니다. GraphQL API를 통해 직관적인 인터페이스를 제공하며, 다양한 ML 모델을 통합할 수 있습니다.

**주요 특징**
- GraphQL API 지원
- 스키마 기반 데이터 구조
- 다양한 벡터화 모듈 지원
- 실시간 데이터 인덱싱
- 하이브리드 검색 (벡터 + 키워드)
- 분산 클러스터링 지원

**참고링크**

- [Weaviate LangChain 문서](https://python.langchain.com/docs/integrations/vectorstores/weaviate/)
- [Weaviate 공식문서](https://weaviate.io/developers/weaviate)
- [LangChain 지원 VectorStore 리스트](https://python.langchain.com/v0.2/docs/integrations/vectorstores/)

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
try:
    from langchain_teddynote import logging
    # 프로젝트 이름을 입력합니다.
    logging.langsmith("CH09-VectorStores-Weaviate")
    print("✅ LangSmith with teddynote 설정 완료")
except ImportError:
    print("⚠️ langchain-teddynote 패키지를 찾을 수 없습니다.")
    print("다음 명령으로 설치하세요: uv add langchain-teddynote")
    
    # 대안: 표준 LangSmith 설정
    import os
    # os.environ["LANGCHAIN_TRACING_V2"] = "true"
    # os.environ["LANGCHAIN_PROJECT"] = "CH09-VectorStores-Weaviate"
    print("✅ 표준 LangSmith 설정으로 대체")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH09-VectorStores-Weaviate
✅ LangSmith with teddynote 설정 완료


## 패키지 설치

Weaviate 사용에 필요한 패키지를 설치합니다.

## 중요 안내: Weaviate v4 API 사용

이 노트북은 최신 Weaviate v4 클라이언트 라이브러리를 사용합니다. 

**주요 변경사항:**
- `weaviate.Client()` → `weaviate.connect_to_local()` / `weaviate.connect_to_weaviate_cloud()`
- `client.schema` → `client.collections`
- GraphQL 쿼리 방식 변경
- 향상된 성능과 안정성

만약 v3 클라이언트를 사용해야 하는 경우:
```bash
pip install "weaviate-client>=3.26.7,<4.0.0"
```

In [3]:
# 필요한 패키지 설치 확인
try:
    import weaviate
    import langchain_weaviate
    import langchain_openai
    import langchain_community
    print("✅ 모든 필요한 패키지가 설치되어 있습니다:")
    print(f"- weaviate-client: {weaviate.__version__}")
    print(f"- langchain-weaviate: 설치됨")
    print(f"- langchain-openai: 설치됨")
    print(f"- langchain-community: 설치됨")
except ImportError as e:
    print(f"❌ 패키지 설치가 필요합니다: {e}")
    print("다음 명령으로 설치하세요:")
    print("pip install weaviate-client langchain-weaviate langchain-openai langchain-community")

✅ 모든 필요한 패키지가 설치되어 있습니다:
- weaviate-client: 4.17.0
- langchain-weaviate: 설치됨
- langchain-openai: 설치됨
- langchain-community: 설치됨


## 샘플 데이터셋 로드

샘플 데이터셋을 로드합니다.

In [4]:
from langchain_community.document_loaders import TextLoader
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_weaviate.vectorstores import WeaviateVectorStore
import weaviate
import weaviate.classes as wvc
import os

# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=0)

# 텍스트 파일을 load -> List[Document] 형태로 변환
loader1 = TextLoader("data/nlp-keywords.txt")
loader2 = TextLoader("data/finance-keywords.txt")

# 문서 분할
split_doc1 = loader1.load_and_split(text_splitter)
split_doc2 = loader2.load_and_split(text_splitter)

# 문서 개수 확인
print(f"NLP 문서 수: {len(split_doc1)}")
print(f"Finance 문서 수: {len(split_doc2)}")
print(f"총 문서 수: {len(split_doc1) + len(split_doc2)}")

NLP 문서 수: 11
Finance 문서 수: 6
총 문서 수: 17


## VectorStore 생성

Weaviate 벡터스토어를 생성합니다. 로컬 Docker 인스턴스와 클라우드 서비스 두 가지 방법을 제공합니다.

### 1. 로컬 Weaviate 설정 (Docker)

Docker를 사용하여 로컬에서 Weaviate를 실행할 수 있습니다.

In [5]:
def create_local_weaviate_connection(url="http://localhost:8080"):
    """
    로컬 Weaviate 서버에 연결하는 함수 (Weaviate v4 API 사용)
    
    Args:
        url (str): Weaviate 서버 URL (기본값: "http://localhost:8080")
    
    Returns:
        tuple: (weaviate.WeaviateClient, WeaviateVectorStore) 또는 (None, None)
    
    Usage:
        # Docker Compose로 Weaviate 서버 실행:
        # 1. docker-compose.yml 파일 생성 (아래 참조)
        # 2. docker-compose up -d
        
        client, vectorstore = create_local_weaviate_connection()
        if vectorstore:
            print("✅ 연결 성공!")
        else:
            print("❌ 연결 실패")
    """
    try:
        # 로컬 Weaviate 서버에 연결 (v4 API)
        client = weaviate.connect_to_local(
            host="localhost",
            port=8080,
            grpc_port=50051
        )
        
        # 서버 상태 확인
        if not client.is_ready():
            raise Exception("Weaviate 서버가 준비되지 않았습니다.")
            
        print(f"✅ Weaviate 서버 연결 성공! (URL: {url})")
        
        # 기존 컬렉션 확인
        collections = client.collections.list_all()
        print(f"📚 기존 컬렉션 수: {len(collections)}")
        
        # 벡터 저장소 생성
        local_weaviate_db = WeaviateVectorStore.from_documents(
            documents=split_doc1,
            embedding=OpenAIEmbeddings(),
            client=client,
            index_name="LocalDocuments",  # 컬렉션명
            text_key="text"
        )
        
        print("✅ 로컬 Weaviate 벡터 저장소 생성 완료")
        return client, local_weaviate_db
        
    except Exception as e:
        print(f"❌ Weaviate 서버 연결 실패: {e}")
        print(f"💡 해결 방법:")
        print(f"   1. Docker Compose로 서버 실행:")
        print(f"      docker-compose up -d")
        print(f"   2. 단일 Docker 컨테이너 실행:")
        print(f"      docker run -p 8080:8080 -p 50051:50051 semitechnologies/weaviate:latest")
        print(f"   3. 웹 대시보드 확인: {url}")
        print(f"   4. Weaviate v4 클라이언트 사용 중")
        return None, None

# Docker Compose 설정 예제 출력
print("📋 Docker Compose 설정  (weaviate/docker-compose.yml):")

# 함수 실행 및 테스트
print("\n🔄 로컬 Weaviate 서버 연결 시도...")
local_client, local_weaviate_db = create_local_weaviate_connection()

# 연결 상태 확인
if local_weaviate_db is not None:
    print("\n🎯 연결 정보:")
    print(f"   • 서버 주소: http://localhost:8080")
    print(f"   • API 엔드포인트: http://localhost:8080/v1")
    print(f"   • 컬렉션명: LocalDocuments")
    print(f"   • 연결 상태: 활성화")
    print(f"   ⚠️  연결을 사용한 후에는 반드시 client.close()를 호출하세요.")
else:
    print("\n⚠️ 로컬 서버가 실행되지 않았습니다.")
    print("위의 Docker Compose 설정을 사용하여 서버를 실행하세요.")

📋 Docker Compose 설정  (weaviate/docker-compose.yml):

🔄 로컬 Weaviate 서버 연결 시도...
✅ Weaviate 서버 연결 성공! (URL: http://localhost:8080)
📚 기존 컬렉션 수: 1
✅ Weaviate 서버 연결 성공! (URL: http://localhost:8080)
📚 기존 컬렉션 수: 1
✅ 로컬 Weaviate 벡터 저장소 생성 완료

🎯 연결 정보:
   • 서버 주소: http://localhost:8080
   • API 엔드포인트: http://localhost:8080/v1
   • 컬렉션명: LocalDocuments
   • 연결 상태: 활성화
   ⚠️  연결을 사용한 후에는 반드시 client.close()를 호출하세요.
✅ 로컬 Weaviate 벡터 저장소 생성 완료

🎯 연결 정보:
   • 서버 주소: http://localhost:8080
   • API 엔드포인트: http://localhost:8080/v1
   • 컬렉션명: LocalDocuments
   • 연결 상태: 활성화
   ⚠️  연결을 사용한 후에는 반드시 client.close()를 호출하세요.


In [6]:
# 안전한 연결 관리를 위한 Context Manager 사용 예제
from contextlib import contextmanager

@contextmanager
def safe_weaviate_connection(connection_type="local", **kwargs):
    """
    안전한 Weaviate 연결을 위한 Context Manager
    
    Args:
        connection_type (str): "local" 또는 "cloud"
        **kwargs: 연결에 필요한 추가 매개변수
    
    Usage:
        # 로컬 연결
        with safe_weaviate_connection("local") as (client, vectorstore):
            if vectorstore:
                results = vectorstore.similarity_search("query", k=3)
                
        # 클라우드 연결  
        with safe_weaviate_connection("cloud") as (client, vectorstore):
            if vectorstore:
                results = vectorstore.similarity_search("query", k=3)
    """
    client = None
    vectorstore = None
    
    try:
        if connection_type == "local":
            client, vectorstore = create_local_weaviate_connection()
        elif connection_type == "cloud":
            client, vectorstore = create_cloud_weaviate_connection()
        else:
            raise ValueError(f"지원하지 않는 연결 타입: {connection_type}")
            
        yield client, vectorstore
        
    except Exception as e:
        print(f"❌ Context Manager에서 오류 발생: {e}")
        yield None, None
        
    finally:
        # 연결 안전하게 종료
        if client is not None:
            try:
                client.close()
                print("🔒 Weaviate 연결이 안전하게 종료되었습니다.")
            except Exception as close_error:
                print(f"⚠️ 연결 종료 중 오류: {close_error}")

print("✅ 안전한 연결 관리 Context Manager 정의 완료")
print("💡 사용법: with safe_weaviate_connection('local') as (client, vectorstore):")

✅ 안전한 연결 관리 Context Manager 정의 완료
💡 사용법: with safe_weaviate_connection('local') as (client, vectorstore):


### 2. Weaviate Cloud Services (WCS) 설정

Weaviate Cloud Services를 사용하여 클라우드에서 Weaviate를 사용할 수 있습니다.

In [ ]:
def create_cloud_weaviate_connection():
    """
    Weaviate Cloud Services에 연결하는 함수 (Weaviate v4 API 사용)
    
    환경변수 필요:
        WEAVIATE_URL: WCS 클러스터 URL
        WEAVIATE_API_KEY: WCS API 키
    
    Returns:
        tuple: (weaviate.WeaviateClient, WeaviateVectorStore) 또는 (None, None)
    """
    try:
        # 환경변수에서 설정값 가져오기
        weaviate_url = os.getenv("WEAVIATE_URL")
        weaviate_api_key = os.getenv("WEAVIATE_API_KEY")
        
        if not weaviate_url or not weaviate_api_key:
            raise ValueError("WEAVIATE_URL 또는 WEAVIATE_API_KEY 환경변수가 설정되지 않았습니다.")
        
        # Weaviate Cloud Services에 연결 (v4 API)
        client = weaviate.connect_to_weaviate_cloud(
            cluster_url=weaviate_url,
            auth_credentials=weaviate.auth.AuthApiKey(weaviate_api_key),
            headers={
                "X-OpenAI-Api-Key": os.getenv("OPENAI_API_KEY")
            }
        )
        
        # 서버 상태 확인
        if not client.is_ready():
            raise Exception("Weaviate 클라우드 서버가 준비되지 않았습니다.")
            
        print(f"✅ Weaviate Cloud 서버 연결 성공!")
        print(f"🌐 클러스터 URL: {weaviate_url}")
        
        # 기존 컬렉션 확인
        collections = client.collections.list_all()
        print(f"📚 기존 컬렉션 수: {len(collections)}")
        
        # 벡터 저장소 생성
        cloud_weaviate_db = WeaviateVectorStore.from_documents(
            documents=split_doc1,
            embedding=OpenAIEmbeddings(),
            client=client,
            index_name="CloudDocuments",  # 컬렉션명
            text_key="text"
        )
        
        print("✅ Weaviate Cloud 벡터 저장소 생성 완료")
        return client, cloud_weaviate_db
        
    except Exception as e:
        print(f"❌ Weaviate Cloud 서버 연결 실패: {e}")
        print(f"💡 해결 방법:")
        print(f"   1. WCS 계정 생성: https://console.weaviate.cloud/")
        print(f"   2. 클러스터 생성 후 URL 및 API 키 획득")
        print(f"   3. .env 파일에 다음 설정 추가:")
        print(f"      WEAVIATE_URL=https://your-cluster.weaviate.network")
        print(f"      WEAVIATE_API_KEY=your-api-key")
        print(f"   4. Weaviate v4 클라이언트 사용 중")
        return None, None

# 클라우드 연결 시도 (환경변수가 설정된 경우에만)
if os.getenv("WEAVIATE_URL") and os.getenv("WEAVIATE_API_KEY"):
    print("🔄 Weaviate Cloud 서버 연결 시도...")
    cloud_client, cloud_weaviate_db = create_cloud_weaviate_connection()
else:
    print("⚠️ Weaviate Cloud 환경변수가 설정되지 않았습니다.")
    print("로컬 서버를 사용하거나 .env 파일에 WCS 설정을 추가하세요.")
    cloud_client, cloud_weaviate_db = None, None

## 기본 검색

Weaviate를 사용한 기본적인 유사도 검색을 수행합니다.

In [7]:
# 안전한 방식으로 검색 수행
print("🔍 Context Manager를 사용한 안전한 검색...")

# 이전 방식은 메모리 누수 위험이 있으므로 사용하지 않습니다
# 대신 Context Manager를 항상 사용합니다

with safe_weaviate_connection("local") as (safe_client, safe_vectorstore):
    if safe_vectorstore is not None:
        print("✅ 로컬 Weaviate 안전한 연결 성공")
        
        # 검색 쿼리
        query = "자연어 처리와 머신러닝"
        
        # 유사도 검색
        results = safe_vectorstore.similarity_search(query, k=3)
        
        print(f"\n📝 검색 쿼리: '{query}'")
        print(f"🎯 검색 결과 ({len(results)}개):")
        print("=" * 50)
        
        for i, doc in enumerate(results, 1):
            print(f"\n[{i}] {doc.page_content[:100]}...")
            print(f"    Source: {doc.metadata.get('source', 'Unknown')}")
            
        print("\n✅ 검색 완료 - 연결이 자동으로 안전하게 종료됩니다")
        
        # 전역 변수 설정 (호환성을 위해)
        weaviate_db = safe_vectorstore
        client = safe_client
        
    else:
        print("❌ Weaviate 서버에 연결할 수 없어 검색을 수행할 수 없습니다.")
        weaviate_db = None
        client = None

print("\n" + "="*60)

🔍 Context Manager를 사용한 안전한 검색...
✅ Weaviate 서버 연결 성공! (URL: http://localhost:8080)
📚 기존 컬렉션 수: 1
✅ 로컬 Weaviate 벡터 저장소 생성 완료
✅ 로컬 Weaviate 안전한 연결 성공
✅ 로컬 Weaviate 벡터 저장소 생성 완료
✅ 로컬 Weaviate 안전한 연결 성공

📝 검색 쿼리: '자연어 처리와 머신러닝'
🎯 검색 결과 (3개):

[1] 정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: W...
    Source: data/nlp-keywords.txt

[2] 정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: W...
    Source: data/nlp-keywords.txt

[3] 정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: W...
    Source: data/nlp-keywords.txt

✅ 검색 완료 - 연결이 자동으로 안전하게 종료됩니다
🔒 Weaviate 연결이 안전하게 종료되었습니다.


📝 검색 쿼리: '자연어 처리와 머신러닝'
🎯 검색 결과 (3개):

[1] 정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: W...
    Source: data/nlp-keywords.txt

[2] 정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡

In [8]:
# Context Manager를 사용한 안전한 검색 예제
print("🔍 Context Manager를 사용한 안전한 검색...")

with safe_weaviate_connection("local") as (safe_client, safe_vectorstore):
    if safe_vectorstore is not None:
        print("✅ 안전한 연결로 검색 수행")
        
        # 검색 쿼리
        query = "자연어 처리와 머신러닝"
        
        # 유사도 검색
        results = safe_vectorstore.similarity_search(query, k=3)
        
        print(f"\n📝 검색 쿼리: '{query}'")
        print(f"🎯 검색 결과 ({len(results)}개):")
        print("=" * 50)
        
        for i, doc in enumerate(results, 1):
            print(f"\n[{i}] {doc.page_content[:100]}...")
            print(f"    Source: {doc.metadata.get('source', 'Unknown')}")
            
        print("\n✅ 검색 완료 및 연결 자동 종료")
    else:
        print("❌ 안전한 연결 생성 실패")

print("\n" + "="*60)

🔍 Context Manager를 사용한 안전한 검색...
✅ Weaviate 서버 연결 성공! (URL: http://localhost:8080)
📚 기존 컬렉션 수: 1
✅ 로컬 Weaviate 벡터 저장소 생성 완료
✅ 안전한 연결로 검색 수행
✅ 로컬 Weaviate 벡터 저장소 생성 완료
✅ 안전한 연결로 검색 수행

📝 검색 쿼리: '자연어 처리와 머신러닝'
🎯 검색 결과 (3개):

[1] 정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: W...
    Source: data/nlp-keywords.txt

[2] 정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: W...
    Source: data/nlp-keywords.txt

[3] 정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: W...
    Source: data/nlp-keywords.txt

✅ 검색 완료 및 연결 자동 종료
🔒 Weaviate 연결이 안전하게 종료되었습니다.


📝 검색 쿼리: '자연어 처리와 머신러닝'
🎯 검색 결과 (3개):

[1] 정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: W...
    Source: data/nlp-keywords.txt

[2] 정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.
예시: W...
    Sour

## 점수와 함께 검색

유사도 점수와 함께 검색 결과를 확인합니다.

## ⚠️ 중요: 연결 생명주기 관리

**문제**: `WeaviateClosedClientError`가 발생하는 이유

Context Manager(`with` 문)를 사용하면 블록이 끝날 때 자동으로 `client.close()`가 호출됩니다. 이후에 닫힌 클라이언트를 사용하려고 하면 `WeaviateClosedClientError`가 발생합니다.

**해결책**: 모든 Weaviate 작업을 Context Manager 내에서 수행하세요.

```python
# ❌ 잘못된 방법
with safe_weaviate_connection("local") as (client, vectorstore):
    pass  # 여기서 연결이 닫힘

vectorstore.similarity_search("query")  # ❌ 오류 발생!

# ✅ 올바른 방법  
with safe_weaviate_connection("local") as (client, vectorstore):
    results = vectorstore.similarity_search("query")  # ✅ 정상 작동
```

In [10]:
if weaviate_db is not None:
    print("🔍 점수와 함께 유사도 검색 수행...")
    
    # ⚠️ 주의: Context Manager 밖에서는 연결이 이미 닫혔습니다.
    # 대신 안전한 Context Manager를 사용합니다.
    
    print("🔄 안전한 연결로 점수 검색 재시도...")
    
    with safe_weaviate_connection("local") as (safe_client, safe_vectorstore):
        if safe_vectorstore is not None:
            # 점수와 함께 검색
            query = "딥러닝과 인공지능"
            results_with_scores = safe_vectorstore.similarity_search_with_score(query, k=3)
            
            print(f"\n📝 검색 쿼리: '{query}'")
            print(f"🎯 검색 결과 (점수 포함):")
            print("=" * 60)
            
            for i, (doc, score) in enumerate(results_with_scores, 1):
                print(f"\n[{i}] 유사도 점수: {score:.4f}")
                print(f"    내용: {doc.page_content[:150]}...")
                print(f"    소스: {doc.metadata.get('source', 'Unknown')}")
                
            print("\n✅ 점수 검색 완료 - 연결이 자동으로 안전하게 종료됩니다")
        else:
            print("❌ 안전한 연결 생성 실패")
        
else:
    print("\n⚠️ Weaviate 서버에 연결할 수 없어 검색을 수행할 수 없습니다.")

🔍 점수와 함께 유사도 검색 수행...
🔄 안전한 연결로 점수 검색 재시도...
✅ Weaviate 서버 연결 성공! (URL: http://localhost:8080)
📚 기존 컬렉션 수: 1
✅ 로컬 Weaviate 벡터 저장소 생성 완료
✅ 로컬 Weaviate 벡터 저장소 생성 완료

📝 검색 쿼리: '딥러닝과 인공지능'
🎯 검색 결과 (점수 포함):

[1] 유사도 점수: 0.8238
    내용: 정의: 멀티모달은 여러 종류의 데이터 모드(예: 텍스트, 이미지, 소리 등)를 결합하여 처리하는 기술입니다. 이는 서로 다른 형식의 데이터 간의 상호 작용을 통해 보다 풍부하고 정확한 정보를 추출하거나 예측하는 데 사용됩니다.
예시: 이미지와 설명 텍스트를 함께 분석하...
    소스: data/nlp-keywords.txt

[2] 유사도 점수: 0.8238
    내용: 정의: 멀티모달은 여러 종류의 데이터 모드(예: 텍스트, 이미지, 소리 등)를 결합하여 처리하는 기술입니다. 이는 서로 다른 형식의 데이터 간의 상호 작용을 통해 보다 풍부하고 정확한 정보를 추출하거나 예측하는 데 사용됩니다.
예시: 이미지와 설명 텍스트를 함께 분석하...
    소스: data/nlp-keywords.txt

[3] 유사도 점수: 0.8238
    내용: 정의: 멀티모달은 여러 종류의 데이터 모드(예: 텍스트, 이미지, 소리 등)를 결합하여 처리하는 기술입니다. 이는 서로 다른 형식의 데이터 간의 상호 작용을 통해 보다 풍부하고 정확한 정보를 추출하거나 예측하는 데 사용됩니다.
예시: 이미지와 설명 텍스트를 함께 분석하...
    소스: data/nlp-keywords.txt

✅ 점수 검색 완료 - 연결이 자동으로 안전하게 종료됩니다
🔒 Weaviate 연결이 안전하게 종료되었습니다.

📝 검색 쿼리: '딥러닝과 인공지능'
🎯 검색 결과 (점수 포함):

[1] 유사도 점수: 0.8238
    내용: 정의: 멀티모달은 여러 종류의 데이터 모드(예: 텍스트, 

## 추가 문서 삽입

기존 벡터스토어에 새로운 문서를 추가합니다.

In [11]:
if weaviate_db is not None:
    print("📄 추가 문서 삽입...")
    
    # ⚠️ 주의: Context Manager 밖에서는 연결이 이미 닫혔습니다.
    # 대신 안전한 Context Manager를 사용합니다.
    
    print("🔄 안전한 연결로 문서 추가 재시도...")
    
    with safe_weaviate_connection("local") as (safe_client, safe_vectorstore):
        if safe_vectorstore is not None:
            # Finance 관련 문서 추가
            safe_vectorstore.add_documents(split_doc2)
            
            print(f"✅ {len(split_doc2)}개의 Finance 문서가 추가되었습니다.")
            
            # 추가된 문서로 검색 테스트
            finance_query = "금융 시장과 투자"
            finance_results = safe_vectorstore.similarity_search(finance_query, k=3)
            
            print(f"\n🔍 Finance 관련 검색: '{finance_query}'")
            print("=" * 50)
            
            for i, doc in enumerate(finance_results, 1):
                print(f"\n[{i}] {doc.page_content[:100]}...")
                print(f"    Source: {doc.metadata.get('source', 'Unknown')}")
                
            print("\n✅ 문서 추가 및 검색 완료 - 연결이 자동으로 안전하게 종료됩니다")
        else:
            print("❌ 안전한 연결 생성 실패")
        
else:
    print("\n⚠️ Weaviate 서버에 연결할 수 없어 문서 추가를 수행할 수 없습니다.")

📄 추가 문서 삽입...
🔄 안전한 연결로 문서 추가 재시도...
✅ Weaviate 서버 연결 성공! (URL: http://localhost:8080)
📚 기존 컬렉션 수: 1
✅ 로컬 Weaviate 벡터 저장소 생성 완료
✅ 로컬 Weaviate 벡터 저장소 생성 완료
✅ 6개의 Finance 문서가 추가되었습니다.
✅ 6개의 Finance 문서가 추가되었습니다.

🔍 Finance 관련 검색: '금융 시장과 투자'

[1] 정의: 시장 가중치는 특정 기업이나 섹터가 전체 지수에서 차지하는 비중을 나타냅니다.
예시: 기술 섹터는 S&P 500 지수에서 가장 큰 시장 가중치를 차지하고 있습니다.
연관키워...
    Source: data/finance-keywords.txt

[2] 정의: 시장 가중치는 특정 기업이나 섹터가 전체 지수에서 차지하는 비중을 나타냅니다.
예시: 기술 섹터는 S&P 500 지수에서 가장 큰 시장 가중치를 차지하고 있습니다.
연관키워...
    Source: data/finance-keywords.txt

[3] 정의: 시장 가중치는 특정 기업이나 섹터가 전체 지수에서 차지하는 비중을 나타냅니다.
예시: 기술 섹터는 S&P 500 지수에서 가장 큰 시장 가중치를 차지하고 있습니다.
연관키워...
    Source: data/finance-keywords.txt

✅ 문서 추가 및 검색 완료 - 연결이 자동으로 안전하게 종료됩니다
🔒 Weaviate 연결이 안전하게 종료되었습니다.

🔍 Finance 관련 검색: '금융 시장과 투자'

[1] 정의: 시장 가중치는 특정 기업이나 섹터가 전체 지수에서 차지하는 비중을 나타냅니다.
예시: 기술 섹터는 S&P 500 지수에서 가장 큰 시장 가중치를 차지하고 있습니다.
연관키워...
    Source: data/finance-keywords.txt

[2] 정의: 시장 가중치는 특정 기업이나 섹터가 전체 지수에서 차지하는 비중을 나타냅니다.
예시: 기술 섹터는 S&P 500 지수에

## 하이브리드 검색

Weaviate의 하이브리드 검색 기능을 사용하여 벡터 검색과 키워드 검색을 결합합니다.

In [13]:
if client is not None:
    print("🔍 Weaviate 하이브리드 검색...")
    
    # ⚠️ 주의: Context Manager 밖에서는 연결이 이미 닫혔습니다.
    # 대신 안전한 Context Manager를 사용합니다.
    
    print("🔄 안전한 연결로 하이브리드 검색 재시도...")
    
    with safe_weaviate_connection("local") as (safe_client, safe_vectorstore):
        if safe_client is not None:
            try:
                # 하이브리드 검색 (벡터 + 키워드) - Weaviate v4 API
                query = "machine learning algorithms"
                
                # 컬렉션 가져오기
                collection_name = "LocalDocuments"
                
                try:
                    collection = safe_client.collections.get(collection_name)
                    
                    # 하이브리드 검색 수행
                    response = collection.query.hybrid(
                        query=query,
                        alpha=0.7,  # 벡터 검색과 키워드 검색의 비율
                        limit=3
                    )
                    
                    print(f"\n📝 하이브리드 검색 쿼리: '{query}'")
                    print(f"🎯 검색 결과:")
                    print("=" * 50)
                    
                    if response.objects:
                        for i, obj in enumerate(response.objects, 1):
                            text = obj.properties.get('text', '')
                            print(f"\n[{i}] {text[:150]}...")
                    else:
                        print("검색 결과가 없습니다.")
                        
                    print("\n✅ 하이브리드 검색 완료 - 연결이 자동으로 안전하게 종료됩니다")
                        
                except Exception as collection_error:
                    print(f"❌ 컬렉션 '{collection_name}' 접근 실패: {collection_error}")
                    print("💡 컬렉션이 존재하지 않거나 데이터가 없을 수 있습니다.")
                    
                    # 대안: 기본 LangChain 검색 사용
                    if safe_vectorstore is not None:
                        print("🔄 기본 검색으로 대체...")
                        results = safe_vectorstore.similarity_search(query, k=3)
                        for i, doc in enumerate(results, 1):
                            print(f"\n[{i}] {doc.page_content[:150]}...")
                
            except Exception as e:
                print(f"❌ 하이브리드 검색 실행 중 오류: {e}")
                print("💡 Weaviate v4 API를 사용 중입니다. 서버가 하이브리드 검색을 지원하는지 확인하세요.")
        else:
            print("❌ 안전한 연결 생성 실패")
        
else:
    print("\n⚠️ Weaviate 클라이언트가 연결되지 않아 하이브리드 검색을 수행할 수 없습니다.")

🔍 Weaviate 하이브리드 검색...
🔄 안전한 연결로 하이브리드 검색 재시도...
✅ Weaviate 서버 연결 성공! (URL: http://localhost:8080)
📚 기존 컬렉션 수: 1
✅ 로컬 Weaviate 벡터 저장소 생성 완료
❌ 컬렉션 'LocalDocuments' 접근 실패: Query call with protocol GRPC search failed with message VectorFromInput was called without vectorizer on class LocalDocuments for input machine learning algorithms.
💡 컬렉션이 존재하지 않거나 데이터가 없을 수 있습니다.
🔄 기본 검색으로 대체...
✅ 로컬 Weaviate 벡터 저장소 생성 완료
❌ 컬렉션 'LocalDocuments' 접근 실패: Query call with protocol GRPC search failed with message VectorFromInput was called without vectorizer on class LocalDocuments for input machine learning algorithms.
💡 컬렉션이 존재하지 않거나 데이터가 없을 수 있습니다.
🔄 기본 검색으로 대체...

[1] 정의: 멀티모달은 여러 종류의 데이터 모드(예: 텍스트, 이미지, 소리 등)를 결합하여 처리하는 기술입니다. 이는 서로 다른 형식의 데이터 간의 상호 작용을 통해 보다 풍부하고 정확한 정보를 추출하거나 예측하는 데 사용됩니다.
예시: 이미지와 설명 텍스트를 함께 분석하...

[2] 정의: 멀티모달은 여러 종류의 데이터 모드(예: 텍스트, 이미지, 소리 등)를 결합하여 처리하는 기술입니다. 이는 서로 다른 형식의 데이터 간의 상호 작용을 통해 보다 풍부하고 정확한 정보를 추출하거나 예측하는 데 사용됩니다.
예시: 이미지와 설명 텍스트를 함께 분석하...

[3] 정의: 멀티모달은 여러 종류의 데이터 모

## 스키마 정보 확인

Weaviate의 스키마 정보를 확인합니다.

In [16]:
if client is not None:
    print("📋 Weaviate 컬렉션 정보...")
    
    # ⚠️ 주의: Context Manager 밖에서는 연결이 이미 닫혔습니다.
    # 대신 안전한 Context Manager를 사용합니다.
    
    print("🔄 안전한 연결로 컬렉션 정보 조회 재시도...")
    
    with safe_weaviate_connection("local") as (safe_client, safe_vectorstore):
        if safe_client is not None:
            try:
                # 전체 컬렉션 가져오기 (Weaviate v4 API)
                collections = safe_client.collections.list_all()
                
                print("\n🏗️ 컬렉션 정보:")
                print("=" * 40)
                
                if collections:
                    for collection_name in collections:
                        try:
                            collection = safe_client.collections.get(collection_name)
                            config = collection.config.get()
                            
                            print(f"\n📁 컬렉션: {collection_name}")
                            print(f"   벡터라이저: {config.vectorizer}")
                            
                            # 속성 정보
                            if hasattr(config, 'properties') and config.properties:
                                print(f"   속성 수: {len(config.properties)}")
                                print("   속성 목록:")
                                for prop_name, prop_config in config.properties.items():
                                    prop_type = prop_config.data_type if hasattr(prop_config, 'data_type') else 'Unknown'
                                    print(f"     - {prop_name} ({prop_type})")
                            else:
                                print("   속성 정보 없음")
                            
                            # 객체 수 확인
                            try:
                                response = collection.aggregate.over_all(total_count=True)
                                count = response.total_count
                                print(f"   총 객체 수: {count}")
                            except Exception as count_error:
                                print(f"   객체 수 확인 실패: {count_error}")
                                
                        except Exception as collection_error:
                            print(f"\n📁 컬렉션: {collection_name}")
                            print(f"   상세 정보 조회 실패: {collection_error}")
                else:
                    print("컬렉션이 없습니다.")
                    
                print("\n✅ 컬렉션 정보 조회 완료 - 연결이 자동으로 안전하게 종료됩니다")
                
            except Exception as e:
                print(f"❌ 컬렉션 정보 조회 중 오류: {e}")
                print("💡 Weaviate v4 API를 사용 중입니다.")
        else:
            print("❌ 안전한 연결 생성 실패")
        
else:
    print("\n⚠️ Weaviate 클라이언트가 연결되지 않아 컬렉션 정보를 확인할 수 없습니다.")

📋 Weaviate 컬렉션 정보...
🔄 안전한 연결로 컬렉션 정보 조회 재시도...
✅ Weaviate 서버 연결 성공! (URL: http://localhost:8080)
📚 기존 컬렉션 수: 1
✅ 로컬 Weaviate 벡터 저장소 생성 완료

🏗️ 컬렉션 정보:

📁 컬렉션: LocalDocuments
   벡터라이저: Vectorizers.NONE
   속성 수: 2
   속성 목록:

📁 컬렉션: LocalDocuments
   상세 정보 조회 실패: 'list' object has no attribute 'items'

✅ 컬렉션 정보 조회 완료 - 연결이 자동으로 안전하게 종료됩니다
🔒 Weaviate 연결이 안전하게 종료되었습니다.
✅ 로컬 Weaviate 벡터 저장소 생성 완료

🏗️ 컬렉션 정보:

📁 컬렉션: LocalDocuments
   벡터라이저: Vectorizers.NONE
   속성 수: 2
   속성 목록:

📁 컬렉션: LocalDocuments
   상세 정보 조회 실패: 'list' object has no attribute 'items'

✅ 컬렉션 정보 조회 완료 - 연결이 자동으로 안전하게 종료됩니다
🔒 Weaviate 연결이 안전하게 종료되었습니다.


## Retriever로 사용

Weaviate VectorStore를 LangChain Retriever로 사용합니다.

In [ ]:
if weaviate_db is not None:
    print("🔄 Retriever로 사용...")
    
    # ⚠️ 주의: Context Manager 밖에서는 연결이 이미 닫혔습니다.
    # 대신 안전한 Context Manager를 사용합니다.
    
    print("🔄 안전한 연결로 Retriever 사용 재시도...")
    
    with safe_weaviate_connection("local") as (safe_client, safe_vectorstore):
        if safe_vectorstore is not None:
            # Retriever 생성
            retriever = safe_vectorstore.as_retriever(
                search_type="similarity",
                search_kwargs={"k": 4}
            )
            
            # Retriever 사용
            query = "데이터 분석과 시각화"
            retrieved_docs = retriever.invoke(query)
            
            print(f"\n📝 Retriever 쿼리: '{query}'")
            print(f"🎯 검색된 문서 ({len(retrieved_docs)}개):")
            print("=" * 50)
            
            for i, doc in enumerate(retrieved_docs, 1):
                print(f"\n[{i}] {doc.page_content[:120]}...")
                print(f"    Source: {doc.metadata.get('source', 'Unknown')}")
                
            print("\n✅ Retriever 사용 완료 - 연결이 자동으로 안전하게 종료됩니다")
        else:
            print("❌ 안전한 연결 생성 실패")
        
else:
    print("\n⚠️ Weaviate 벡터스토어가 사용 불가능합니다.")

🔄 Retriever로 사용...
🔄 안전한 연결로 Retriever 사용 재시도...
✅ Weaviate 서버 연결 성공! (URL: http://localhost:8080)
📚 기존 컬렉션 수: 1
✅ 로컬 Weaviate 벡터 저장소 생성 완료
✅ 로컬 Weaviate 벡터 저장소 생성 완료

📝 Retriever 쿼리: '데이터 분석과 시각화'
🎯 검색된 문서 (4개):

[1] 정의: DataFrame은 행과 열로 이루어진 테이블 형태의 데이터 구조로, 주로 데이터 분석 및 처리에 사용됩니다.
예시: 판다스 라이브러리에서 DataFrame은 다양한 데이터 타입의 열을 가질 수 있으며, 데이...
    Source: data/nlp-keywords.txt

[2] 정의: DataFrame은 행과 열로 이루어진 테이블 형태의 데이터 구조로, 주로 데이터 분석 및 처리에 사용됩니다.
예시: 판다스 라이브러리에서 DataFrame은 다양한 데이터 타입의 열을 가질 수 있으며, 데이...
    Source: data/nlp-keywords.txt

[3] 정의: DataFrame은 행과 열로 이루어진 테이블 형태의 데이터 구조로, 주로 데이터 분석 및 처리에 사용됩니다.
예시: 판다스 라이브러리에서 DataFrame은 다양한 데이터 타입의 열을 가질 수 있으며, 데이...
    Source: data/nlp-keywords.txt

[4] 정의: DataFrame은 행과 열로 이루어진 테이블 형태의 데이터 구조로, 주로 데이터 분석 및 처리에 사용됩니다.
예시: 판다스 라이브러리에서 DataFrame은 다양한 데이터 타입의 열을 가질 수 있으며, 데이...
    Source: data/nlp-keywords.txt

✅ Retriever 사용 완료 - 연결이 자동으로 안전하게 종료됩니다
🔒 Weaviate 연결이 안전하게 종료되었습니다.

📝 Retriever 쿼리: '데이터 분석과 시각화'
🎯 검색된 문서 (4개):

[1] 정의: DataFrame은

Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"API key has expired"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"API key has expired"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"API key has expired"}\n')


## 정리

생성된 클래스나 데이터를 정리합니다.

In [18]:
def cleanup_weaviate_collection(client, collection_name):
    """
    Weaviate 컬렉션을 삭제하는 함수 (Weaviate v4 API)
    
    Args:
        client: Weaviate 클라이언트
        collection_name (str): 삭제할 컬렉션명
    """
    try:
        # 컬렉션 존재 확인
        collections = client.collections.list_all()
        
        if collection_name in collections:
            client.collections.delete(collection_name)
            print(f"✅ 컬렉션 '{collection_name}' 삭제 완료")
        else:
            print(f"⚠️ 컬렉션 '{collection_name}'이 존재하지 않습니다.")
            
    except Exception as e:
        print(f"❌ 컬렉션 삭제 실패: {e}")

def close_weaviate_connection(client):
    """
    Weaviate 연결을 안전하게 종료하는 함수
    """
    try:
        if client is not None:
            client.close()
            print("✅ Weaviate 연결이 안전하게 종료되었습니다.")
    except Exception as e:
        print(f"❌ 연결 종료 중 오류: {e}")

# 정리 실행 (필요한 경우 주석 해제)
# if client is not None:
#     print("🧹 리소스 정리...")
#     cleanup_weaviate_collection(client, "LocalDocuments")
#     cleanup_weaviate_collection(client, "CloudDocuments")
#     close_weaviate_connection(client)
# else:
#     print("⚠️ 클라이언트가 연결되지 않아 정리를 수행할 수 없습니다.")

# 현재 세션의 연결 정리 (메모리 누수 방지)
if 'local_client' in locals() and local_client is not None:
    print("🧹 현재 세션의 로컬 연결 정리...")
    try:
        local_client.close()
        print("✅ 로컬 Weaviate 연결이 종료되었습니다.")
    except Exception as e:
        print(f"⚠️ 로컬 연결 종료 중 오류: {e}")

if 'cloud_client' in locals() and cloud_client is not None:
    print("🧹 현재 세션의 클라우드 연결 정리...")
    try:
        cloud_client.close()
        print("✅ 클라우드 Weaviate 연결이 종료되었습니다.")
    except Exception as e:
        print(f"⚠️ 클라우드 연결 종료 중 오류: {e}")

print("\n✨ Weaviate 벡터스토어 튜토리얼 완료!")
print("\n📚 추가 학습 자료:")
print("   • Weaviate 공식 문서: https://weaviate.io/developers/weaviate")
print("   • LangChain Weaviate 통합: https://python.langchain.com/docs/integrations/vectorstores/weaviate/")
print("   • Weaviate GraphQL API: https://weaviate.io/developers/weaviate/api/graphql")
print("   • Weaviate Cloud Services: https://console.weaviate.cloud/")

🧹 현재 세션의 로컬 연결 정리...
✅ 로컬 Weaviate 연결이 종료되었습니다.

✨ Weaviate 벡터스토어 튜토리얼 완료!

📚 추가 학습 자료:
   • Weaviate 공식 문서: https://weaviate.io/developers/weaviate
   • LangChain Weaviate 통합: https://python.langchain.com/docs/integrations/vectorstores/weaviate/
   • Weaviate GraphQL API: https://weaviate.io/developers/weaviate/api/graphql
   • Weaviate Cloud Services: https://console.weaviate.cloud/
